# Detectron2 (Original+ResNeSt)

<img src="https://dl.fbaipublicfiles.com/detectron2/Detectron2-Logo-Horz.png" width="500">

This is a modification of the offiical detectron2 colab that can be found [here](https://colab.research.google.com/drive/16jcaJoc6bCFAQ96jDe2HwtXj7BMD_-m5 ).

This colab is a combination of [facebookresearch/detectron2](https://github.com/facebookresearch/detectron2) and [zhanghang1989/detectron2-ResNeSt](https://github.com/zhanghang1989/detectron2-ResNeSt).

My fork is located in [styler00dollar/Colab-Detectron2-with-Original-and-ResNeSt](https://github.com/styler00dollar/Colab-Detectron2-with-Original-and-ResNeSt).


In [ ]:
!nvidia-smi

# Install detectron2

In [ ]:
#@title Install dependencies
# install dependencies: (use cu101 because colab has CUDA 10.1)
!pip install -U torch==1.5 torchvision==0.6 -f https://download.pytorch.org/whl/cu101/torch_stable.html 
!pip install cython pyyaml==5.1
!pip install -U 'git+https://github.com/cocodataset/cocoapi.git#subdirectory=PythonAPI'
import torch, torchvision
print(torch.__version__, torch.cuda.is_available())
!gcc --version
# opencv is pre-installed on colab

# Install detectron2 or detectron2-ResNeSt

In [ ]:
#@title Install detectron2
# install detectron2:
!pip install detectron2==0.1.3 -f https://dl.fbaipublicfiles.com/detectron2/wheels/cu101/index.html
# git clone detectron2
!git clone https://github.com/facebookresearch/detectron2

In [ ]:
#@title Install detectron2-ResNeSt

# install detectron resnest
!python -m pip install 'git+https://github.com/zhanghang1989/detectron2-ResNeSt.git'
import detectron2

# clone detectron2 resnest
%cd /content/
!git clone https://github.com/zhanghang1989/detectron2-ResNeSt

In [ ]:
#@title Import detectron2

# You may need to restart your runtime prior to this, to let your installation take effect
# Some basic setup:
# Setup detectron2 logger
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common libraries
import numpy as np
import cv2
import random
from google.colab.patches import cv2_imshow

# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog

# Run a pre-trained detectron2 model

In [ ]:
!wget http://images.cocodataset.org/val2017/000000439715.jpg -O input.jpg
im = cv2.imread("./input.jpg")
cv2_imshow(im)

In [ ]:
cfg = get_cfg()
# add project-specific config (e.g., TensorMask) here if you're not running a model in detectron2's core library
cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5  # set threshold for this model
# Find a model from detectron2's model zoo. You can use the https://dl.fbaipublicfiles... url as well
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml")
predictor = DefaultPredictor(cfg)
outputs = predictor(im)

In [ ]:
# look at the outputs. See https://detectron2.readthedocs.io/tutorials/models.html#model-output-format for specification
outputs["instances"].pred_classes
outputs["instances"].pred_boxes

In [ ]:
# We can use `Visualizer` to draw the predictions on the image.
v = Visualizer(im[:, :, ::-1], MetadataCatalog.get(cfg.DATASETS.TRAIN[0]), scale=1.2)
v = v.draw_instance_predictions(outputs["instances"].to("cpu"))
cv2_imshow(v.get_image()[:, :, ::-1])

# Train on a custom coco dataset

Copy a .7z file and extract its contents.

The datastructure should be like this:
 - some_name.7z
  - train
    - image1.png
    - lksjfdl.png
    - ...
    - train.json
  - test
    - test1.png
    - lksdjfl.png
    - ...
    - test.json

In [ ]:
# connect Google Drive if you want to copy dataset
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# copy to local storage and extract
!cp -r '/content/drive/My Drive/webp.7z' /content/dataset_new_coco.7z
%cd /content/
!7z x /content/dataset_new_coco.7z -oc:/content/

## Train config


In [ ]:
#@title Register dataset
from detectron2.data.datasets import register_coco_instances
register_coco_instances("my_dataset_train2", {}, "/content/train/train.json", "/content/train")
register_coco_instances("my_dataset_val2", {}, "/content/test/test.json", "/content/test")

In [ ]:
#@title Activate tensorboard
# Look at training curves in tensorboard:
%load_ext tensorboard
%tensorboard --logdir "/content/drive/My Drive/detectron2"

In [ ]:
import os
import numpy as np
import json
from detectron2.structures import BoxMode
from detectron2.engine import DefaultTrainer
from detectron2.config import get_cfg

cfg = get_cfg()
cfg.OUTPUT_DIR = '/content/drive/My Drive/detectron2'
os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)

# select config
#cfg.merge_from_file("/content/detectron2-ResNeSt/configs/COCO-InstanceSegmentation/mask_cascade_rcnn_ResNeSt_50_FPN_syncBN_1x.yaml")
#cfg.merge_from_file("/content/detectron2-ResNeSt/configs/COCO-InstanceSegmentation/mask_cascade_rcnn_ResNeSt_101_FPN_syncBN_1x.yaml")

cfg.merge_from_file("/content/detectron2/configs/COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_1x.yaml")

cfg.DATASETS.TRAIN = ("my_dataset_train2",)
cfg.DATASETS.TEST = ("my_dataset_val2",)
cfg.DATALOADER.NUM_WORKERS = 2

#select config again
#cfg.MODEL.WEIGHTS = ("/content/detectron2-ResNeSt/configs/COCO-InstanceSegmentation/mask_cascade_rcnn_ResNeSt_50_FPN_syncBN_1x.yaml") # Let training ini  tialize from model zoo
#cfg.MODEL.WEIGHTS = ("/content/detectron2-ResNeSt/configs/COCO-InstanceSegmentation/mask_cascade_rcnn_ResNeSt_101_FPN_syncBN_1x.yaml") # Let training ini  tialize from model zoo

cfg.MODEL.WEIGHTS = ("/content/detectron2/configs/COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_1x.yaml")

cfg.INPUT.MAX_SIZE_TEST = 1600
cfg.INPUT.MAX_SIZE_TRAIN = 1600
cfg.SOLVER.IMS_PER_BATCH = 3   # increase this for more speed, but it will need more vram
cfg.SOLVER.BASE_LR = 0.00025  # pick a good LR
cfg.SOLVER.MAX_ITER = 500000    # 300 iterations seems good enough for this toy dataset; you may need to train longer for a practical dataset
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 4096   # or 128 for example
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 2  # only has one class (ballon)
cfg.SNAPSHOT_ITERS = 5000 # when checkpoint gets created

trainer = DefaultTrainer(cfg) 
#trainer.resume_or_load(resume=True)
trainer.train()

## Inference & evaluation using the trained model
Now, let's run inference with the trained model on the balloon validation dataset. First, let's create a predictor using the model we just trained:



In [ ]:
cfg.MODEL.WEIGHTS = '/content/drive/My Drive/detectron2/model_final.pth'
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.80   # set the testing threshold for this model
cfg.DATASETS.TEST = ("my_dataset_val2", )
predictor = DefaultPredictor(cfg)

Then, we randomly select several samples to visualize the prediction results.

In [ ]:
from detectron2.utils.visualizer import ColorMode
balloon_metadata = MetadataCatalog.get("my_dataset_train2")

#im = cv2.imread("/content/Tensorflow-Object-Detection-API-train-custom-Mask-R-CNN-model/images/train/IMG_20181228_101826.jpg")

im = cv2.imread("/content/test/fec6805603202500617d027a2b28df39.png")
outputs = predictor(im)
v = Visualizer(im[:, :, ::-1],
                metadata=balloon_metadata, 
                scale=0.5, 
                instance_mode=ColorMode.IMAGE_BW   # remove the colors of unsegmented pixels. This option is only available for segmentation models
)
v = v.draw_instance_predictions(outputs["instances"].to("cpu"))
cv2_imshow(v.get_image()[:, :, ::-1])


We can also evaluate its performance using AP metric implemented in COCO API.

In [ ]:
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.data import build_detection_test_loader
evaluator = COCOEvaluator("my_dataset_val2", cfg, False, output_dir="./output/")
val_loader = build_detection_test_loader(cfg, "my_dataset_val2")
inference_on_dataset(trainer.model, val_loader, evaluator)
# another equivalent way is to use trainer.test